# SLKB Pipeline

Here, we will go over the discussed pipeline using a Toy Data. Feel free to use this file to analyze your dataset. The file is divided into 3 main parts: (1) Data creation, (2) Score calculation, (3) Query Results

## Before getting started

Make sure an R environment with GEMINI, and mageck tool are located in your path. To see whether you can run their respective scores or not, you can run the following command:

```
import shutil
shutil.which('R') ## should yield accessed R environment location
shutil.which('mageck') ## should yield MAGeCK location
```

In additon, make sure to install SLKB python package. The details can be located at its [website](https://www.google.com)



In [ ]:
## First, we load in our packages
import SLKB

import numpy as np
import pandas as pd
import os
import pickle
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import subprocess
import shlex


# Section 1 - Data Preperation

First, we start by installing a pickle file that contains the toy data. Not all input files are required. For score calculation, only sequences and counts files are sufficient. 

In [ ]:
# taken from

sequences_ref = urllib.request.urlopen(pickle_loc).read()
counts_ref = urllib.request.urlopen(pickle_loc).read()
scores_ref = urllib.request.urlopen(pickle_loc).read()

# let us create a local sqlite3 database to store our results in, and connect to it

db_engine = create_SLKB(location = os.getcwd(), name = 'myCDKO_db')
SLKB_engine = sqlalchemy.create_engine(db_engine)

In [ ]:
print(counts_ref.head(15))

In [ ]:
print(sequences_ref.head(15))

In [ ]:
print(scores_ref.head(15))

## Inserting to DB

After each data is prepared, the study is ready to be inserted into the database. The ```prepare_study_for_export``` function will go over the data and prepare the data for insertion. It will produce errors where necessary, make sure that your files match with the template. 

Make sure your control gene list is set up properly to correctly categorize the counts file. The counts file will produce a ```target_type``` column that contains three categories:
1. Dual - Both sgRNAs targeting different genes.
2. Single - Both sgRNAs targeting the same gene (i.e., gene_1 + gene_1, or gene_1 + control)
3. Control - Both sgRNAs targeting controls.

In [ ]:
study_controls = ['CONTROL']
study_conditions = [['T0_rep1', 'T0_rep2', 'T0_rep3'],
                    ['TEnd_rep1', 'TEnd_rep2', 'TEnd_rep3']]

db_inserts = prepare_study_for_export(sequence_ref = sequence_ref.copy(), 
                                      counts_ref = counts_ref.copy(),
                                      scores_ref = scores_ref.copy(),
                                      study_controls = study_controls,
                                      study_conditions = study_conditions)

print(db_inserts['score_ref'].head(15))
print(db_inserts['sequences_ref'].head(15))
print(db_inserts['counts_ref'].head(15))

In [ ]:
# Finally, insert the data to the database
insert_study_to_db(SLKB_engine, db_inserts)

# Section 2 - Score Calculation

Here, we calculate the scores and add them to the database. First, we start by querying the data we just deposited.

In [ ]:
# read the data

# experiment design
experiment_design = pd.read_sql_table('CDKO_EXPERIMENT_DESIGN', SLKB_engine, index_col = 'sgRNA_id')
experiment_design.reset_index(drop = True, inplace = True)
experiment_design.index.rename('sgRNA_id', inplace = True)

# counts
counts = pd.read_sql_table('CDKO_SGRNA_COUNTS', SLKB_engine, index_col = 'sgRNA_pair_id')
counts.reset_index(drop = True, inplace = True)
counts.index.rename('sgRNA_pair_id', inplace = True)

# scores
scores = pd.read_sql_table('CDKO_ORIGINAL_SL_RESULTS', SLKB_engine, index_col = 'id')
scores.reset_index(drop = True, inplace = True)
scores.index.rename('gene_pair_id', inplace = True)

# join the tables together
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_1_id', right_index = True, suffixes = ('', '_g1'))
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_2_id', right_index = True, suffixes = ('', '_g2'))
# rename
counts = counts.rename({'sgRNA_guide_name': 'sgRNA_guide_name_g1',
                        'sgRNA_guide_seq': 'sgRNA_guide_seq_g1',
                        'sgRNA_target_name': 'sgRNA_target_name_g1',
                        'study_origin_x': 'study_origin',
                        'cell_line_origin_x': 'cell_line_origin'}, axis = 1)


curr_counts = counts[(counts['study_origin'] == 'myStudy') & (counts['cell_line_origin'] == 'myCL')]

## Median B/NB Score

In [ ]:
if check_if_added_to_table(curr_counts.copy(), 'MEDIAN_NB_SCORE', SLKB_engine):
    median_res = run_median_scores(curr_counts.copy())
    add_table_to_db(curr_counts.copy(), median_res['MEDIAN_NB_SCORE'], 'MEDIAN_NB_SCORE', SLKB_engine)
    if median_res['MEDIAN_B_SCORE'] is not None:
        add_table_to_db(curr_counts.copy(), median_res['MEDIAN_B_SCORE'], 'MEDIAN_B_SCORE', SLKB_engine)

## sgRNA Derived B/NB Score

In [ ]:
if not check_if_added_to_table(curr_counts.copy(), 'SGRA_DERIVED_NB_SCORE', SLKB_engine):
    sgRNA_res = run_sgrna_scores(curr_counts.copy())
    add_table_to_db(curr_counts.copy(), sgRNA_res['SGRA_DERIVED_NB_SCORE'], 'SGRA_DERIVED_NB_SCORE', SLKB_engine)
    if sgRNA_res['SGRA_DERIVED_B_SCORE'] is not None:
        add_table_to_db(curr_counts.copy(), sgRNA_res['SGRA_DERIVED_B_SCORE'], 'SGRA_DERIVED_B_SCORE', SLKB_engine)

## Horlbeck Score

In [ ]:
if not check_if_added_to_table(curr_counts.copy(), 'HORLBECK_SCORE', SLKB_engine):
    horlbeck_res = run_horlbeck_score(curr_counts.copy(), curr_study = curr_study, curr_cl = curr_cl, store_loc = os.getcwd(), save_dir = 'HORLBECK_Files', do_preprocessing = True, re_run = False)
    add_table_to_db(curr_counts.copy(), horlbeck_res['HORLBECK_SCORE'], 'HORLBECK_SCORE', SLKB_engine)

## GEMINI Score

In [ ]:
cmd_params = ['module load R/4.1.0']
if not check_if_added_to_table(curr_counts.copy(), 'GEMINI_SCORE', SLKB_engine):
    gemini_res = run_gemini_score(curr_counts.copy(), curr_study = curr_study, curr_cl = curr_cl, store_loc = os.getcwd(), save_dir = 'GEMINI_Files', command_line_params = cmd_params, re_run = False)
    add_table_to_db(curr_counts.copy(), gemini_res['GEMINI_SCORE'], 'GEMINI_SCORE', SLKB_engine)

## MAGeCK Score

In [ ]:
cmd_params = ['conda activate myEnv']
if not check_if_added_to_table(curr_counts.copy(), 'MAGECK_SCORE', SLKB_engine):
    mageck_res = run_mageck_score(curr_counts.copy(), curr_study = curr_study, curr_cl = curr_cl, store_loc = os.getcwd(), save_dir = 'MAGECK_Files', command_line_params = cmd_params,re_run = False)
    add_table_to_db(curr_counts.copy(), mageck_res['MAGECK_SCORE'], 'MAGECK_SCORE', SLKB_engine)

# Section 3 - Query Results

Finally, we can query the data to produce the calculation table.

![table](../../docs/images/Calculation.png)

In [ ]:
# read the data

# experiment design
experiment_design = pd.read_sql_table('CDKO_EXPERIMENT_DESIGN', SLKB_engine, index_col = 'sgRNA_id')
experiment_design.drop(['study_origin'], axis = 1, inplace = True)
experiment_design.reset_index(drop = True, inplace = True)
experiment_design.index.rename('sgRNA_id', inplace = True)

# counts
counts = pd.read_sql_table('CDKO_SGRNA_COUNTS', SLKB_engine, index_col = 'sgRNA_pair_id')
counts.reset_index(drop = True, inplace = True)
counts.index.rename('sgRNA_pair_id', inplace = True)

# scores
scores = pd.read_sql_table('CDKO_ORIGINAL_SL_RESULTS', SLKB_engine, index_col = 'gene_pair_id')

# join the tables together
counts = counts.merge(scores, how = 'left', left_on = 'gene_pair_id', right_index = True)
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_1_id', right_index = True, suffixes = ('', '_g1'))
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_2_id', right_index = True, suffixes = ('', '_g2'))
# rename
counts = counts.rename({'sgRNA_guide_name': 'sgRNA_guide_name_g1',
                        'sgRNA_guide_seq': 'sgRNA_guide_seq_g1',
                        'sgRNA_target_name': 'sgRNA_target_name_g1',
                        'study_origin_x': 'study_origin',
                        'cell_line_origin_x': 'cell_line_origin'}, axis = 1)

experiment_design = pd.read_sql_table('CDKO_EXPERIMENT_DESIGN', SLKB_engine, index_col = 'sgRNA_id')
experiment_design.reset_index(drop = True, inplace = True)
experiment_design.index.rename('sgRNA_id', inplace = True)

# tables to obtain the data from
all_results_tables = ['HORLBECK_SCORE', 
                      'MAGECK_SCORE', 
                      'MEDIAN_NB_SCORE', 
                      'MEDIAN_B_SCORE', 
                      'SGRA_DERIVED_NB_SCORE', 
                      'SGRA_DERIVED_B_SCORE', 
                      'GEMINI_SCORE']

#################

all_scores = []
for curr_study in available_studies:
    print('Working on study: ' + curr_study)

    # get study counts and seq
    study_counts = counts.loc[counts['study_origin'] == study_name_to_pubmed_id[curr_study]].copy()

    curr_seq_ids = np.array(sorted(list(set(study_counts['guide_1_id'].tolist() + study_counts['guide_2_id'].tolist()))))
    study_sequences = experiment_design.loc[curr_seq_ids]

    # the analysis runs for each individual cell line
    available_cell_lines = set(study_counts['cell_line_origin'])


    for curr_cl in available_cell_lines:
        # store results here
        study_scores = []
    
        print('Working on cell line: ' + curr_cl)
        curr_counts = study_counts.loc[study_counts['cell_line_origin'] == curr_cl].copy()
        
        for table_name in all_results_tables:
            # add the result of the table to the list
            study_scores.append(query_result_table(curr_counts.copy(), table_name, curr_study, curr_cl, SLKB_engine))
    
        # remove duplicate annotation columns
        study_scores = pd.concat(study_scores, axis = 1, ignore_index = False)
        study_scores = study_scores.loc[:,~study_scores.columns.duplicated(keep = 'last')].copy()
        
        # make sure the annotations are all filled
        study_scores['gene_pair'] = study_scores.index
        study_scores['study_origin'] = curr_study
        study_scores['cell_line_origin'] = curr_cl
        
        # reset the index, gene_pair -> id
        study_scores.reset_index(drop = True, inplace = True)

        # add to big table 
        all_scores.append(study_scores)
    
    print('-----')
    
print('Done getting all data!')
    
# combine the scores at the end
all_scores = pd.concat(all_scores, axis = 0, ignore_index = True)

# add individual genes
all_scores['gene_1'] = [i.split('|')[0] for i in all_scores['gene_pair']]
all_scores['gene_2'] = [i.split('|')[1] for i in all_scores['gene_pair']]

# sort such that all annotations are at the front
all_columns = sorted(list(all_scores.columns))
annotation_columns = ['gene_pair', 'gene_1', 'gene_2', 'study_origin', 'cell_line_origin']

# get the final scores
all_scores = all_scores.loc[:, annotation_columns + [i for i in all_columns if i not in annotation_columns]]

In [ ]:
# save to current directory
all_scores.to_csv(os.path.join(os.getcwd(), 'all_scores.csv'))